<a href="https://colab.research.google.com/github/LumaLynai/ProjetoFinalETL/blob/main/ProjetoFinalETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bibliotecas**

In [1]:
!pip install pandas
!pip install gcsfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 7.4 MB/s 
     |████████████████████████████████| 140 kB 66.9 MB/s 
     |████████████████████████████████| 271 kB 62.3 MB/s 
     |████████████████████████████████| 144 kB 35.7 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 


In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 31 kB/s 
     |████████████████████████████████| 198 kB 51.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=d58b990040cde09fba0b0f620f0946466a29d9d75d829ef2e320eba8b9d55c02
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [3]:
import pandas as pd
import psycopg2
import requests
import pymongo
import gcsfs
import os 
import csv

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [4]:
from google.cloud import storage

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.window import *
from pyspark.sql.types import *
from pyspark import SparkConf
import pyspark.sql.functions as F 
import pyspark.sql.window as w

# **GCP**
Conectando backet

In [8]:
serviceAccount = r'/content/drive/MyDrive/arquivos/macro-mercury-349020-d9ed9a670580 (1).json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [9]:
client = storage.Client()
bucket = client.get_bucket('soulcode-bc17-telecom')

#**Meu_Municio_Acesso.csv**

```
```



##**Pandas**


In [94]:
df1_extrai = pd.read_csv('https://storage.googleapis.com/telecomunicacao2/Meu_Municipio_Acessos.csv',sep=';')

**Copy** 

In [95]:
df1_pd = df1_extrai.copy()

### Inserindo no Backet 
Original

In [152]:
from pandas._libs import index
df1_extrai.to_csv('gs://soulcode-bc17-telecom/Original/Meu_Municipio_Acessos.csv',index=False)


In [ ]:
df1_pd.head(2)

### Info

In [26]:
df1_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66837 entries, 0 to 66836
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Ano              66837 non-null  int64 
 1   Mês              66837 non-null  int64 
 2   Acessos          66837 non-null  int64 
 3   Serviço          66837 non-null  object
 4   Densidade        66837 non-null  object
 5   Código IBGE      66837 non-null  int64 
 6   Município        66837 non-null  object
 7   UF               66837 non-null  object
 8   Nome UF          66837 non-null  object
 9   Região           66837 non-null  object
 10  Código Nacional  66837 non-null  int64 
dtypes: int64(5), object(6)
memory usage: 5.6+ MB


### Unique

In [44]:
df1_pd['Município'].unique()

array(['Paulo Frontin - PR', 'Araruama - RJ', 'Pinhão - PR', ...,
       'Ribeiro Gonçalves - PI', 'Itacurubi - RS', 'Carrapateira - PB'],
      dtype=object)

In [99]:
df1=df1_pd['Município'].str[:-5]

In [103]:
df1_pd['Município']=df1_pd['Município'].str[:-5]

In [104]:
df1_pd.head()

,Ano,Mês,Acessos,Serviço,Densidade,Código IBGE,Município,UF,Nome UF,Região,Código Nacional
0,2021,6,5597,Telefonia Móvel,"73,114931636659",4118709,Paulo Frontin,PR,Paraná,Sul,42
1,2021,6,713,Banda Larga Fixa,"27,993718099725",4118709,Paulo Frontin,PR,Paraná,Sul,42
2,2021,6,132,TV por Assinatura,"5,1825677267373",4118709,Paulo Frontin,PR,Paraná,Sul,42
3,2021,6,327,Telefonia Fixa,"12,83863368669",4118709,Paulo Frontin,PR,Paraná,Sul,42
4,2021,6,162149,Telefonia Móvel,"113,77138048893",3300209,Araruama,RJ,Rio de Janeiro,Sudeste,22


##**Spark**

###SessionSpark


In [123]:
spark = SparkSession.builder.appName("pandas em spark").getOrCreate()

In [124]:
spark

In [125]:
df1_spark = spark.createDataFrame(df1_pd)

In [107]:
df1_spark.show()

+----+---+-------+-----------------+----------------+-----------+-----------------+---+--------------+--------+---------------+
| Ano|Mês|Acessos|          Serviço|       Densidade|Código IBGE|        Município| UF|       Nome UF|  Região|Código Nacional|
+----+---+-------+-----------------+----------------+-----------+-----------------+---+--------------+--------+---------------+
|2021|  6|   5597|  Telefonia Móvel| 73,114931636659|    4118709|    Paulo Frontin| PR|        Paraná|     Sul|             42|
|2021|  6|    713| Banda Larga Fixa| 27,993718099725|    4118709|    Paulo Frontin| PR|        Paraná|     Sul|             42|
|2021|  6|    132|TV por Assinatura| 5,1825677267373|    4118709|    Paulo Frontin| PR|        Paraná|     Sul|             42|
|2021|  6|    327|   Telefonia Fixa|  12,83863368669|    4118709|    Paulo Frontin| PR|        Paraná|     Sul|             42|
|2021|  6| 162149|  Telefonia Móvel| 113,77138048893|    3300209|         Araruama| RJ|Rio de Janeiro| S

In [82]:
df1_spark.select('Código Nacional').distinct().count()

67

In [83]:
df1_spark.select('Código Nacional').distinct().show()

+---------------+
|Código Nacional|
+---------------+
|             65|
|             19|
|             54|
|             22|
|             77|
|             34|
|             94|
|             43|
|             32|
|             84|
|             31|
|             98|
|             95|
|             71|
|             68|
|             87|
|             63|
|             27|
|             51|
|             17|
+---------------+
only showing top 20 rows



In [111]:
df1_spark.select('Município').distinct().show()

+--------------------+
|           Município|
+--------------------+
|            Araruama|
|        Borrazópolis|
|              Utinga|
|              Maruim|
|             Ibiporã|
|          Livramento|
|    Senador Guiomard|
|           Fronteira|
|              Pongaí|
| Leopoldo de Bulhões|
|               Apodi|
|      Benedito Leite|
|   Tiradentes do Sul|
|        Assis Brasil|
|  Ribas do Rio Pardo|
|Santo Antônio dos...|
|Boa Esperança do Sul|
|           Carrancas|
|         Piranguinho|
|            Rio Novo|
+--------------------+
only showing top 20 rows



### Like

In [113]:
df1_spark.filter(F.col('Município').like("%Santo Antônio dos%")).show()

+----+---+-------+-----------------+---------------+-----------+--------------------+---+--------+--------+---------------+
| Ano|Mês|Acessos|          Serviço|      Densidade|Código IBGE|           Município| UF| Nome UF|  Região|Código Nacional|
+----+---+-------+-----------------+---------------+-----------+--------------------+---+--------+--------+---------------+
|2021|  6|  11480|  Telefonia Móvel|77,558187577469|    2110302|Santo Antônio dos...| MA|Maranhão|Nordeste|             99|
|2021|  6|     54| Banda Larga Fixa|1,3015184381779|    2110302|Santo Antônio dos...| MA|Maranhão|Nordeste|             99|
|2021|  6|    428|TV por Assinatura|10,315738732225|    2110302|Santo Antônio dos...| MA|Maranhão|Nordeste|             99|
|2021|  6|     46|   Telefonia Fixa|1,1087008917812|    2110302|Santo Antônio dos...| MA|Maranhão|Nordeste|             99|
|2021|  6|   1914|  Telefonia Móvel|87,165281625115|    2209450|Santo Antônio dos...| PI|   Piauí|Nordeste|             86|
|2021|  

In [126]:
df1_spark.filter(F.col('Município').like("%Couto de Magalhãe%")).show()

+----+---+-------+-----------------+---------------+-----------+--------------------+---+------------+-------+---------------+
| Ano|Mês|Acessos|          Serviço|      Densidade|Código IBGE|           Município| UF|     Nome UF| Região|Código Nacional|
+----+---+-------+-----------------+---------------+-----------+--------------------+---+------------+-------+---------------+
|2021|  6|   3390|  Telefonia Móvel|73,841284196247|    3120102|Couto de Magalhãe...| MG|Minas Gerais|Sudeste|             38|
|2021|  6|    372| Banda Larga Fixa|25,237449118046|    3120102|Couto de Magalhãe...| MG|Minas Gerais|Sudeste|             38|
|2021|  6|    177|TV por Assinatura|12,008141112619|    3120102|Couto de Magalhãe...| MG|Minas Gerais|Sudeste|             38|
|2021|  6|     91|   Telefonia Fixa|6,1736770691995|    3120102|Couto de Magalhãe...| MG|Minas Gerais|Sudeste|             38|
|2020| 12|   2598|  Telefonia Móvel|58,738412841962|    3120102|Couto de Magalhãe...| MG|Minas Gerais|Sudeste| 

In [87]:
df1_spark.filter(F.col('Município').like("%MG%")).count()

10235

### Drop coluna
* Código Nacional
* Nome UF

In [137]:
df1_spark=df1_spark.drop(F.col('Código Nacional')).drop(F.col(' Município'))

In [139]:
df1_spark.show(1)

+----+---+-------+---------------+---------------+-----------+-------------+---+-------+------+
| Ano|Mês|Acessos|        Serviço|      Densidade|Código IBGE|    Município| UF|Nome UF|Região|
+----+---+-------+---------------+---------------+-----------+-------------+---+-------+------+
|2021|  6|   5597|Telefonia Móvel|73,114931636659|    4118709|Paulo Frontin| PR| Paraná|   Sul|
+----+---+-------+---------------+---------------+-----------+-------------+---+-------+------+
only showing top 1 row



###witcolumnRenamed
* Mês == Mes
* Serviço == Servico
* Código IBGE == Cod_IBGE
* Município == Municipio
* Região == Regiao

In [142]:
df1_spark=(df1_spark.withColumnRenamed("Mês","Mes")
         .withColumnRenamed("Serviço","Servico")
         .withColumnRenamed("Código IBGE","Cod_IBGE")
         .withColumnRenamed("Município","Municipio")
         .withColumnRenamed("Região","Regiao")
)

In [143]:
df1_spark.show()

+----+---+-------+-----------------+----------------+--------+-----------------+---+--------------+--------+
| Ano|Mes|Acessos|          Servico|       Densidade|Cod_IBGE|        Municipio| UF|       Nome UF|  Regiao|
+----+---+-------+-----------------+----------------+--------+-----------------+---+--------------+--------+
|2021|  6|   5597|  Telefonia Móvel| 73,114931636659| 4118709|    Paulo Frontin| PR|        Paraná|     Sul|
|2021|  6|    713| Banda Larga Fixa| 27,993718099725| 4118709|    Paulo Frontin| PR|        Paraná|     Sul|
|2021|  6|    132|TV por Assinatura| 5,1825677267373| 4118709|    Paulo Frontin| PR|        Paraná|     Sul|
|2021|  6|    327|   Telefonia Fixa|  12,83863368669| 4118709|    Paulo Frontin| PR|        Paraná|     Sul|
|2021|  6| 162149|  Telefonia Móvel| 113,77138048893| 3300209|         Araruama| RJ|Rio de Janeiro| Sudeste|
|2021|  6|  32457| Banda Larga Fixa| 67,673734909614| 3300209|         Araruama| RJ|Rio de Janeiro| Sudeste|
|2021|  6|  11287|T

###Inserindo bucket 
Tratados 

In [145]:
df1_spark.toPandas().to_parquet('gs://soulcode-bc17-telecom/Tratados/Meu_Municipio_AcessosTratad.gzip',compression='gzip')

# **Velocidade_Contratada_SCM.csv**


In [149]:
df2_extrai=pd.read_csv('gs://telecomunicacao2/Velocidade_Contratada_SCM.csv',sep=";")

**Copy**

In [150]:
df2_pd = df2_extrai.copy()

###Inserindo Bucket

In [190]:
df2_extrai.to_csv('gs://soulcode-bc17-telecom/Original/Velocidade_Contratada_SCM.csv',index=False)


In [176]:
df2_pd.head(2)

,Ano,Mês,razao_social,cnpj,velocidade_contratada_mbps,uf,municipio,codigo_ibge,acessos,tipo,municipio_uf
0,2020.0,12.0,algar telecom s/a,71.208.516/0001-74,40,MG,UBERLANDIA,3170206.0,16660.0,INTERNET,Uberlândia - MG
1,2020.0,12.0,algar telecom s/a,71.208.516/0001-74,40,SP,FRANCA,3516200.0,5093.0,INTERNET,Franca - SP


###Unique

In [155]:
pd.unique(df2_pd['Mês'])

array([12., 11., 10.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1., nan])

###Info

In [158]:
df2_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5789372 entries, 0 to 5789371
Data columns (total 11 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   Ano                         float64
 1   Mês                         float64
 2   razao_social                object 
 3   cnpj                        object 
 4   velocidade_contratada_mbps  object 
 5   uf                          object 
 6   municipio                   object 
 7   codigo_ibge                 float64
 8   acessos                     float64
 9   tipo                        object 
 10  municipio_uf                object 
dtypes: float64(4), object(7)
memory usage: 485.9+ MB


###Duplicated

In [162]:
df2_pd.duplicated().sum()

11684

In [166]:
df_pd=df2_pd[df2_pd.duplicated()]

In [ ]:
df_pd.head()

In [174]:
df2_pd= df2_pd.drop_duplicates()

In [182]:
df2_pd.count()

Ano                           5777687
Mês                           5777687
razao_social                  5777687
cnpj                          5777687
velocidade_contratada_mbps    5777593
uf                            5777687
municipio                     5777687
codigo_ibge                   5777687
acessos                       5777432
tipo                          5777687
municipio_uf                  5777687
dtype: int64

###Isnull

In [181]:
pd.isnull(df2_pd).sum()

Ano                             1
Mês                             1
razao_social                    1
cnpj                            1
velocidade_contratada_mbps     95
uf                              1
municipio                       1
codigo_ibge                     1
acessos                       256
tipo                            1
municipio_uf                    1
dtype: int64

###Drop

**Coluna**

In [231]:
df2_pd=df2_pd.drop(columns=['municipio_uf'])

In [232]:
df2_pd.head()

,Ano,Mes,razao_social,cnpj,velocidade_contratada_mbps,uf,municipio,Cod_IBGE,acessos,tipo
0,2020.0,12.0,algar telecom s/a,71.208.516/0001-74,40,MG,UBERLANDIA,3170206.0,16660.0,INTERNET
1,2020.0,12.0,algar telecom s/a,71.208.516/0001-74,40,SP,FRANCA,3516200.0,5093.0,INTERNET
2,2020.0,12.0,algar telecom s/a,71.208.516/0001-74,10,MG,UBERLANDIA,3170206.0,10507.0,INTERNET
3,2020.0,12.0,algar telecom s/a,71.208.516/0001-74,60,MG,CAPINOPOLIS,3112604.0,498.0,INTERNET
4,2020.0,12.0,algar telecom s/a,71.208.516/0001-74,100,MG,SANTA JULIANA,3157708.0,508.0,INTERNET


**Linhas** 
* Valores  isnull == 360
* df2_pd == 5777687
* isnull 0.006% == df2_pd

In [213]:
df2_pd=df2_pd.dropna()

In [214]:
pd.isnull(df2_pd).sum()

Ano                           0
Mês                           0
razao_social                  0
cnpj                          0
velocidade_contratada_mbps    0
uf                            0
municipio                     0
codigo_ibge                   0
acessos                       0
tipo                          0
municipio_uf                  0
dtype: int64

###Unique

In [215]:
pd.unique(df2_pd['municipio_uf'])

array(['Uberlândia - MG', 'Franca - SP', 'Capinópolis - MG', ...,
       'Quixaba - PB', 'Quixaba - PE', 'São Valério - TO'], dtype=object)

###Rename 


In [225]:
df2_pd.rename(columns = {'Mês':'Mes','codigo_ibge':'Cod_IBGE'},inplace=True)

In [233]:
df2_pd.head(1)

,Ano,Mes,razao_social,cnpj,velocidade_contratada_mbps,uf,municipio,Cod_IBGE,acessos,tipo
0,2020.0,12.0,algar telecom s/a,71.208.516/0001-74,40,MG,UBERLANDIA,3170206.0,16660.0,INTERNET


###Inserindo bucket 
Tratados

In [235]:
df2_pd.to_parquet('gs://soulcode-bc17-telecom/Tratados/Velocidade_Contratada_Tratad.gzip',compression='gzip')

In [217]:
df2_pd.head(1)

,Ano,Mês,razao_social,cnpj,velocidade_contratada_mbps,uf,municipio,codigo_ibge,acessos,tipo,municipio_uf
0,2020.0,12.0,algar telecom s/a,71.208.516/0001-74,40,MG,UBERLANDIA,3170206.0,16660.0,INTERNET,Uberlândia - MG


# **Acessos_Telefonia_Movel_Pre_Pos_Total.json**

In [10]:
df3_extrai = pd.read_json('gs://soulcode-bc17-telecom/Acessos_Telefonia_Movel_Pre_Pos_Total.json')

In [13]:
df3_pd=df3_extrai.copy()

In [23]:
df3_pd.head()

,Ano,Mês,Modalidade de Cobrança,Acessos
0,2022,3,Pós-pago,138629611
1,2022,3,Pré-pago,119642952
2,2022,2,Pós-pago,137439913
3,2022,2,Pré-pago,119004007
4,2022,1,Pós-pago,136500794


## Duplicated

In [16]:
df3_test=df3_pd[df3_pd.duplicated()]

In [18]:
df3_test.count()

Ano                       0
Mês                       0
Modalidade de Cobrança    0
Acessos                   0
dtype: int64

## Info

In [19]:
df3_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 412 entries, 0 to 411
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Ano                     412 non-null    int64 
 1   Mês                     412 non-null    int64 
 2   Modalidade de Cobrança  412 non-null    object
 3   Acessos                 412 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 16.1+ KB


## Unique 

In [30]:
pd.unique(df3_pd['Mês'])

array([ 3,  2,  1, 12, 11, 10,  9,  8,  7,  6,  5,  4])

## Rename

In [31]:
df3_pd.rename(columns = {'Mês':'Mes','Modalidade de Cobrança':'ModalidadeCobranca'},inplace=True)

In [ ]:
df3_pd.head()

## Inserindo Bucket

In [33]:
df3_pd.to_parquet('gs://soulcode-bc17-telecom/Tratados/Acessos_Telefonia_Movel_Pre_Pos_TotalTratad.gzip',compression='gzip')

#**Densidade_Telefonia_Movel.json**

In [45]:
df4_extrai = pd.read_json('gs://soulcode-bc17-telecom/Densidade_Telefonia_Movel.json')

In [63]:
df4_pd = df4_extrai.copy()

In [64]:
df4_pd.count()

Ano                           223046
Mês                           223046
UF                            222998
Município                     222998
Código IBGE                   222998
Densidade                     222998
Nível Geográfico Densidade    223046
dtype: int64

In [49]:
df4_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 223046 entries, 0 to 223045
Data columns (total 7 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Ano                         223046 non-null  int64  
 1   Mês                         223046 non-null  int64  
 2   UF                          222998 non-null  object 
 3   Município                   222998 non-null  object 
 4   Código IBGE                 222998 non-null  float64
 5   Densidade                   222998 non-null  object 
 6   Nível Geográfico Densidade  223046 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 13.6+ MB


In [65]:
pd.unique(df4_pd['Nível Geográfico Densidade'])

array(['Brasil', 'UF', 'Municipio'], dtype=object)

In [55]:
df4_test=df4_pd[df4_pd.duplicated()]

In [56]:
df4_test.count()

Ano                           0
Mês                           0
UF                            0
Município                     0
Código IBGE                   0
Densidade                     0
Nível Geográfico Densidade    0
dtype: int64

## Rename

In [66]:
df4_pd.rename(columns = {'Mês':'Mes','Município':'Municipio','Código IBGE':'Cod_IBGE','Nível Geográfico Densidade':'NivelGeografico'},inplace=True)

In [67]:
df4_pd.head()

,Ano,Mes,UF,Municipio,Cod_IBGE,Densidade,NivelGeografico
0,2022,3,Brasil,Brasil,0.0,"121,96723524202",Brasil
1,2022,2,Brasil,Brasil,0.0,"121,10367262288",Brasil
2,2022,1,Brasil,Brasil,0.0,"120,73751906513",Brasil
3,2021,12,Brasil,Brasil,0.0,"120,28536970803",Brasil
4,2021,11,Brasil,Brasil,0.0,"119,48517870301",Brasil


## Inserindo no Bucket 

In [68]:
df4_pd.to_parquet('gs://soulcode-bc17-telecom/Tratados/Densidade_Telefonia_Movel.gzip',compression='gzip')